In [4]:
import numpy as np
import pandas as pd
import bs4
import requests
import csv

In [5]:
base_url = 'https://web.archive.org'

In [8]:
county_df = pd.read_csv('pop_18_poverty.csv')
county_df['Geo_FIPS'] = county_df['Geo_FIPS'].astype(str)
county_df['Geo_FIPS'] = county_df['Geo_FIPS'].str.rjust(5, '0')
county_df['State'] = county_df['Geo_FIPS'].str[:2]

tx_counties = county_df[county_df.State == '48']
tx_counties.drop(['Geo_QName', 'pop_18_in_poverty', 'State'], axis=1, inplace=True)
tx_counties['Name'] = tx_counties['Geo_NAME'].str[:-7]
tx_counties

state_counties = {}
for geoid, _, name in tx_counties.itertuples(index=False):
    state_counties[name] = geoid

state_counties

<ipython-input-8-944f691e7312>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tx_counties['Name'] = tx_counties['Geo_NAME'].str[:-7]


{'Anderson': '48001',
 'Andrews': '48003',
 'Angelina': '48005',
 'Aransas': '48007',
 'Archer': '48009',
 'Armstrong': '48011',
 'Atascosa': '48013',
 'Austin': '48015',
 'Bailey': '48017',
 'Bandera': '48019',
 'Bastrop': '48021',
 'Baylor': '48023',
 'Bee': '48025',
 'Bell': '48027',
 'Bexar': '48029',
 'Blanco': '48031',
 'Borden': '48033',
 'Bosque': '48035',
 'Bowie': '48037',
 'Brazoria': '48039',
 'Brazos': '48041',
 'Brewster': '48043',
 'Briscoe': '48045',
 'Brooks': '48047',
 'Brown': '48049',
 'Burleson': '48051',
 'Burnet': '48053',
 'Caldwell': '48055',
 'Calhoun': '48057',
 'Callahan': '48059',
 'Cameron': '48061',
 'Camp': '48063',
 'Carson': '48065',
 'Cass': '48067',
 'Castro': '48069',
 'Chambers': '48071',
 'Cherokee': '48073',
 'Childress': '48075',
 'Clay': '48077',
 'Cochran': '48079',
 'Coke': '48081',
 'Coleman': '48083',
 'Collin': '48085',
 'Collingsworth': '48087',
 'Colorado': '48089',
 'Comal': '48091',
 'Comanche': '48093',
 'Concho': '48095',
 'Cooke': '

In [9]:
def sub_table_gather(sub_url):
    new_r = requests.get(sub_url)
    new_text = new_r.text
    
    new_soup = bs4.BeautifulSoup(new_text, "html5lib")
    
    return_list = []
    
#     print(len(new_soup.find_all("table")[0].find_all("tr")[1:]))
    for tr_tag in new_soup.find_all("table")[0].find_all("tr")[1:]:
        a_val = tr_tag.find('a').get_text()
        values = tr_tag.find_all('td', class_='text-right')
        cust_tot = values[0].get_text()
        cust_tot = int(''.join(c for c in cust_tot if c.isnumeric()))
        cust_out = values[1].get_text()
        cust_out = int(''.join(c for c in cust_out if c.isnumeric()))
        return_list.append((a_val, cust_tot, cust_out))
    
    return return_list

In [10]:
def county_utility(sub_url):
    new_r = requests.get(sub_url)
    new_text = new_r.text

    new_soup = bs4.BeautifulSoup(new_text, "html5lib")
    return_list = []

    try:
        utility = new_soup.find("h1", id="UtilityName").get_text().strip()
    except:
        return return_list
    for tr_tag in new_soup.find_all("table")[0].find_all("tr")[1:]:
        a_val = tr_tag.find('a').get_text()
#         values = tr_tag.find_all('td', class_='text-right')
#         cust_tot = values[0].get_text()
#         cust_tot = int(''.join(c for c in cust_tot if c.isnumeric()))
    #     cust_out = values[1].get_text()
    #     cust_out = int(''.join(c for c in cust_out if c.isnumeric()))
        return_list.append(a_val)

    return return_list

In [11]:
# url = 'https://web.archive.org/web/20210217175313/poweroutage.us/area/state/texas'
# base_url = 'https://web.archive.org'
# r = requests.get(url)
# text = r.text
# soup = bs4.BeautifulSoup(text, "html5lib")

# counties_at_time = {}

# for tr_tag in soup.find_all("table")[0].find_all("tr")[1:]:
#     utility = tr_tag.find("a").get_text()
#     tracked = tr_tag.find_all("div", class_="text-right col-xs-6 col-md-2")[0].get_text()
#     tracked = int(''.join(c for c in tracked if c.isnumeric()))
#     out = tr_tag.find_all("div", class_="text-right col-xs-6 col-md-2")[1].get_text()
#     out = int(''.join(c for c in out if c.isnumeric()))
    
#     href = tr_tag.find("a").get('href')
#     new_url = base_url + href
#     counties = county_utility(new_url)
# #     print(counties)
#     for county in counties:
#         if county not in state_counties:
#             continue
#         if county not in counties_at_time:
#             counties_at_time[county] = {'total': 0, 'out': 0}
#         counties_at_time[county]['total'] += tracked
#         counties_at_time[county]['out'] += out
        
# counties_at_time

In [12]:
def scrape_utility(url, base_url, state_counties):
    r = requests.get(url)
    text = r.text
    soup = bs4.BeautifulSoup(text, "html5lib")

    counties_at_time = {}

    for tr_tag in soup.find_all("table")[0].find_all("tr")[1:]:
        utility = tr_tag.find("a").get_text()
        tracked = tr_tag.find_all("div", class_="text-right col-xs-6 col-md-2")[0].get_text()
        tracked = int(''.join(c for c in tracked if c.isnumeric()))
        out = tr_tag.find_all("div", class_="text-right col-xs-6 col-md-2")[1].get_text()
        out = int(''.join(c for c in out if c.isnumeric()))

        href = tr_tag.find("a").get('href')
        new_url = base_url + href
        counties = county_utility(new_url)
    #     print(counties)
        for county in counties:
            if county not in state_counties:
                continue
            if county not in counties_at_time:
                counties_at_time[county] = {'total': 0, 'out': 0}
            counties_at_time[county]['total'] += tracked
            counties_at_time[county]['out'] += out

    return counties_at_time
    
    

In [13]:
def scrape_url(url, base_url, state_counties):
    return_dict = {}
    
    r = requests.get(url)
    text = r.text

    soup = bs4.BeautifulSoup(text, "html5lib")
    for a_tag in soup.find_all("table")[0].find_all("a"):
        href = a_tag.get('href')
        new_url = base_url + href
        counties = sub_table_gather(new_url)
        for (county, cust_all, cust_without) in counties:
            if county not in state_counties:
                continue
            if county not in return_dict:
                return_dict[county] = {'total': 0, 'without': 0}
            return_dict[county]['total'] += cust_all
            return_dict[county]['without'] += cust_without
    
    return return_dict

In [14]:
# sun_14_url_1 = 'https://web.archive.org/web/20210214062520/https://poweroutage.us/area/state/texas'
# sun_14 = scrape_url(sun_14_url_1, base_url, state_counties)

In [15]:
# mon_15_url_6 = 'https://web.archive.org/web/20210215125507/https://poweroutage.us/area/state/texas'
# mon_15_6 = scrape_url(mon_15_url_6, base_url, state_counties)

In [16]:
# mon_15_url_21 = 'https://web.archive.org/web/20210216031625/https://poweroutage.us/area/state/texas'
# mon_15_21 = scrape_url(mon_15_url_21, base_url, state_counties)

In [17]:
# tues_16_url_12 = 'https://web.archive.org/web/20210216180701/https://poweroutage.us/area/state/texas'
# tues_16_12 = scrape_url(tues_16_url_12, base_url, state_counties)

In [18]:
# wed_17_url_12 = 'https://web.archive.org/web/20210217175313/https://poweroutage.us/area/state/texas'
# wed_17_12 = scrape_url(wed_17_url_12, base_url, state_counties)

In [19]:
# thur_18_url_12 = 'https://web.archive.org/web/20210218180638/https://poweroutage.us/area/state/texas'
# thur_18_12 = scrape_url(thur_18_url_12, base_url, state_counties)

In [20]:
# with open('outage_data.csv', mode='w') as out_file:
#     out_writer = csv.writer(out_file, delimiter=',')
#     for county, dic in sun_14.items():
#         cust_all = list(dic.values())[0]
#         cust_out = list(dic.values())[1]
#         try:
#             ratio = cust_out/cust_all
#         except:
#             ratio = 0
#         out_writer.writerow([state_counties[county], county, cust_all, cust_out, ratio, 'Sunday'])
#     for county, dic in mon_15_6.items():
#         cust_all = list(dic.values())[0]
#         cust_out = list(dic.values())[1]
#         try:
#             ratio = cust_out/cust_all
#         except:
#             ratio = 0
#         out_writer.writerow([state_counties[county], county, cust_all, cust_out, ratio, 'Monday 6AM'])
#     for county, dic in mon_15_21.items():
#         cust_all = list(dic.values())[0]
#         cust_out = list(dic.values())[1]
#         try:
#             ratio = cust_out/cust_all
#         except:
#             ratio = 0
#         out_writer.writerow([state_counties[county], county, cust_all, cust_out, ratio, 'Monday 9PM'])
#     for county, dic in tues_16_12.items():
#         cust_all = list(dic.values())[0]
#         cust_out = list(dic.values())[1]
#         try:
#             ratio = cust_out/cust_all
#         except:
#             ratio = 0
#         out_writer.writerow([state_counties[county], county, cust_all, cust_out, ratio, 'Tuesday'])
#     for county, dic in wed_17_12.items():
#         cust_all = list(dic.values())[0]
#         cust_out = list(dic.values())[1]
#         try:
#             ratio = cust_out/cust_all
#         except:http://localhost:8889/notebooks/Desktop/data_viz/final_project/web_scraper.ipynb#
#             ratio = 0
#         out_writer.writerow([state_counties[county], county, cust_all, cust_out, ratio, 'Wednesday'])
#     for county, dic in thur_18_12.items():
#         cust_all = list(dic.values())[0]
#         cust_out = list(dic.values())[1]
#         try:
#             ratio = cust_out/cust_all
#         except:
#             ratio = 0
#         out_writer.writerow([state_counties[county], county, cust_all, cust_out, ratio, 'Thursday'])
    

In [21]:
two_fourteen_midnight_url = 'https://web.archive.org/web/20210214062520/poweroutage.us/area/state/texas'
two_fourteen_midnight = scrape_utility(two_fourteen_midnight_url, base_url, state_counties)
two_fourteen_midnight

{'Aransas': {'total': 1062682, 'out': 771},
 'Atascosa': {'total': 1962969, 'out': 835},
 'Bee': {'total': 1062682, 'out': 771},
 'Brewster': {'total': 1051021, 'out': 771},
 'Briscoe': {'total': 1058913, 'out': 771},
 'Brooks': {'total': 1220444, 'out': 821},
 'Brown': {'total': 4839220, 'out': 3791},
 'Caldwell': {'total': 2077825, 'out': 1980},
 'Calhoun': {'total': 1051021, 'out': 771},
 'Callahan': {'total': 1086162, 'out': 852},
 'Cameron': {'total': 1648430, 'out': 823},
 'Childress': {'total': 1051021, 'out': 771},
 'Coke': {'total': 4855517, 'out': 3910},
 'Coleman': {'total': 4839797, 'out': 3872},
 'Colorado': {'total': 3760984, 'out': 2429},
 'Concho': {'total': 4847507, 'out': 3897},
 'Cottle': {'total': 1051021, 'out': 771},
 'Crane': {'total': 4821938, 'out': 3791},
 'Crockett': {'total': 4831787, 'out': 3859},
 'Dickens': {'total': 1051021, 'out': 771},
 'Dimmit': {'total': 1092083, 'out': 771},
 'Duval': {'total': 1111667, 'out': 771},
 'Eastland': {'total': 4887408, '

In [24]:
two_fifteen_6_url = 'https://web.archive.org/web/20210215125507/poweroutage.us/area/state/texas/'
two_fifteen_6 = scrape_utility(two_fifteen_6_url, base_url, state_counties)
two_fifteen_6

{'Aransas': {'total': 1064497, 'out': 274309},
 'Atascosa': {'total': 1970084, 'out': 474509},
 'Bee': {'total': 1064497, 'out': 274309},
 'Brewster': {'total': 1052836, 'out': 274309},
 'Briscoe': {'total': 1060728, 'out': 274309},
 'Brooks': {'total': 1227092, 'out': 314461},
 'Brown': {'total': 4841057, 'out': 1291703},
 'Caldwell': {'total': 2079648, 'out': 472501},
 'Calhoun': {'total': 1052836, 'out': 274309},
 'Callahan': {'total': 1087977, 'out': 277985},
 'Cameron': {'total': 1650235, 'out': 381819},
 'Childress': {'total': 1052836, 'out': 274309},
 'Coke': {'total': 4857278, 'out': 1308202},
 'Coleman': {'total': 4841634, 'out': 1295379},
 'Colorado': {'total': 3762833, 'out': 422333},
 'Concho': {'total': 4849275, 'out': 1305457},
 'Cottle': {'total': 1052836, 'out': 274309},
 'Crane': {'total': 4823775, 'out': 1291703},
 'Crockett': {'total': 4833631, 'out': 1292634},
 'Dickens': {'total': 1052836, 'out': 274309},
 'Dimmit': {'total': 1098731, 'out': 286425},
 'Duval': {'to

In [25]:
two_fifteen_16_url = 'https://web.archive.org/web/20210215222152/poweroutage.us/area/state/texas'
two_fifteen_16 = scrape_utility(two_fifteen_16_url, base_url, state_counties)
two_fifteen_16

{'Aransas': {'total': 1064497, 'out': 346476},
 'Atascosa': {'total': 1958791, 'out': 526926},
 'Bee': {'total': 1064497, 'out': 346476},
 'Brewster': {'total': 1052836, 'out': 346476},
 'Briscoe': {'total': 1060728, 'out': 346476},
 'Brooks': {'total': 1215799, 'out': 384190},
 'Brown': {'total': 4841057, 'out': 1457970},
 'Caldwell': {'total': 2079658, 'out': 604969},
 'Calhoun': {'total': 1052836, 'out': 346476},
 'Callahan': {'total': 1087980, 'out': 350421},
 'Cameron': {'total': 1650235, 'out': 462944},
 'Childress': {'total': 1052836, 'out': 346476},
 'Coke': {'total': 4857295, 'out': 1476080},
 'Coleman': {'total': 4841637, 'out': 1461915},
 'Colorado': {'total': 3762827, 'out': 1540515},
 'Concho': {'total': 4849289, 'out': 1474464},
 'Cottle': {'total': 1052836, 'out': 346476},
 'Crane': {'total': 4823775, 'out': 1457970},
 'Crockett': {'total': 4833631, 'out': 1460299},
 'Dickens': {'total': 1052836, 'out': 346476},
 'Dimmit': {'total': 1087438, 'out': 355340},
 'Duval': {'t

In [26]:
two_sixteen_13_url = 'https://web.archive.org/web/20210216195013/poweroutage.us/area/state/texas'
two_sixteen_13 = scrape_utility(two_sixteen_13_url, base_url, state_counties)
two_sixteen_13

{'Aransas': {'total': 1064497, 'out': 382781},
 'Atascosa': {'total': 1959856, 'out': 684981},
 'Bee': {'total': 1064497, 'out': 382781},
 'Brewster': {'total': 1052836, 'out': 382781},
 'Briscoe': {'total': 1060729, 'out': 382781},
 'Brooks': {'total': 1217621, 'out': 422216},
 'Brown': {'total': 4851117, 'out': 1483250},
 'Caldwell': {'total': 2079651, 'out': 652078},
 'Calhoun': {'total': 1052836, 'out': 382781},
 'Callahan': {'total': 1098040, 'out': 397413},
 'Cameron': {'total': 1650233, 'out': 505461},
 'Childress': {'total': 1052836, 'out': 382781},
 'Coke': {'total': 4857331, 'out': 1491942},
 'Coleman': {'total': 4841635, 'out': 1477834},
 'Colorado': {'total': 3762835, 'out': 1599569},
 'Concho': {'total': 4849326, 'out': 1489759},
 'Cottle': {'total': 1052836, 'out': 382781},
 'Crane': {'total': 4823774, 'out': 1473226},
 'Crockett': {'total': 4833630, 'out': 1475651},
 'Dickens': {'total': 1052836, 'out': 382781},
 'Dimmit': {'total': 1089260, 'out': 396107},
 'Duval': {'t

In [27]:
two_seventeen_11_url = 'https://web.archive.org/web/20210217175313/poweroutage.us/area/state/texas'
two_seventeen_11 = scrape_utility(two_seventeen_11_url, base_url, state_counties)
two_seventeen_11

{'Aransas': {'total': 1064497, 'out': 355230},
 'Atascosa': {'total': 1958992, 'out': 646621},
 'Bee': {'total': 1064497, 'out': 355230},
 'Brewster': {'total': 1052836, 'out': 355230},
 'Briscoe': {'total': 1060729, 'out': 355230},
 'Brooks': {'total': 1217129, 'out': 382293},
 'Brown': {'total': 4851173, 'out': 986314},
 'Caldwell': {'total': 2080149, 'out': 460111},
 'Calhoun': {'total': 1052836, 'out': 355230},
 'Callahan': {'total': 1098098, 'out': 369985},
 'Cameron': {'total': 1650232, 'out': 411546},
 'Childress': {'total': 1052836, 'out': 355230},
 'Coke': {'total': 4850768, 'out': 981331},
 'Coleman': {'total': 4841637, 'out': 981021},
 'Colorado': {'total': 3762806, 'out': 1729293},
 'Concho': {'total': 4842761, 'out': 978687},
 'Cottle': {'total': 1052836, 'out': 355230},
 'Crane': {'total': 4823774, 'out': 976290},
 'Crockett': {'total': 4833630, 'out': 978377},
 'Dickens': {'total': 1052836, 'out': 355230},
 'Dimmit': {'total': 1088768, 'out': 365350},
 'Duval': {'total':

In [28]:
two_eighteen_11_url = 'https://web.archive.org/web/20210218180638/poweroutage.us/area/state/texas'
two_eightteen_11 = scrape_utility(two_eighteen_11_url, base_url, state_counties)
two_eightteen_11

NameError: name 'two_eighteen_11' is not defined

In [22]:
with open('outage_data_2.csv', mode='w') as out_file:
    out_writer = csv.writer(out_file, delimiter=',')
    for county, dic in two_fourteen_midnight.items():
        cust_all = list(dic.values())[0]
        cust_out = list(dic.values())[1]
        try:
            ratio = cust_out/cust_all
        except:
            ratio = 0
        out_writer.writerow([state_counties[county], county, cust_all, cust_out, ratio, 'Sunday'])

In [30]:
final_dict = {}

for (county, geo_fip) in state_counties.items():
    if geo_fip not in final_dict:
        final_dict[geo_fip] = {'name': county, 'ratio_1': 0, 'ratio_2': 0, 'ratio_3': 0, 'ratio_4': 0, 'ratio_5': 0, 'ratio_6': 0}

for (county, duct) in two_fourteen_midnight.items():
    geo_fip = state_counties[county]
    tot = duct['total']
    out = duct['out']
    try:
        ratio = out / tot
    except:
        ratio = 0
    final_dict[geo_fip]['ratio_1'] = ratio

for (county, duct) in two_fifteen_6.items():
    geo_fip = state_counties[county]
    tot = duct['total']
    out = duct['out']
    try:
        ratio = out / tot
    except:
        ratio = 0
    final_dict[geo_fip]['ratio_2'] = ratio

for (county, duct) in two_fifteen_16.items():
    geo_fip = state_counties[county]
    tot = duct['total']
    out = duct['out']
    try:
        ratio = out / tot
    except:
        ratio = 0
    final_dict[geo_fip]['ratio_3'] = ratio

for (county, duct) in two_sixteen_13.items():
    geo_fip = state_counties[county]
    tot = duct['total']
    out = duct['out']
    try:
        ratio = out / tot
    except:
        ratio = 0
    final_dict[geo_fip]['ratio_4'] = ratio
    
for (county, duct) in two_seventeen_11.items():
    geo_fip = state_counties[county]
    tot = duct['total']
    out = duct['out']
    try:
        ratio = out / tot
    except:
        ratio = 0
    final_dict[geo_fip]['ratio_5'] = ratio
    
for (county, duct) in two_eightteen_11.items():
    geo_fip = state_counties[county]
    tot = duct['total']
    out = duct['out']
    try:
        ratio = out / tot
    except:
        ratio = 0
    final_dict[geo_fip]['ratio_56'] = ratio

len(final_dict)

254

In [31]:
import json

with open("outage_utility.json", "w") as outfile:
    json.dump(final_dict, outfile)